# Assignment#2- Hidden Markov Model 
Submitted by:
Akshay Sanjeev. 


A version of this is uploaded to : https://github.com/askay-sanjeev/seqanalysis-homeworks/tree/main/HW-2


In [3]:
import numpy as np
import hw2_fns as HMM
from numpy.random import choice as choose

In [4]:
# from the question
S = ['H', 'T']
T = 3
a = np.array([[  0,  .5,  .5], #transition probabilities
              [.01, .94, .05],
              [.01, .05, .94]])
e = np.array([[.5, .5], #emission probabilities
              [.8, .2]])



## 1. HMM Generator

In [5]:

def hmm_gen(S, T, a, e):
    '''
    This notation will be followed for the rest of the notebook:
    S = observable states. e.g, {Head, Tail}
    T = hidden states. e.g, {0(init/term), 1(Fair), 2(Biased)}
    a = transition probabilites for hidden states
    e = emission probabilities of hidden states
    OUTPUT:
    pi = hidden state chain
    x = observable state chain
    '''
    T =  np.arange(T)
    pi = []
    x = []
    pi.append(choose(T[1:], p=a[0, 1: len(T)]))
    x.append(choose(S, p=e[pi[0]-1, :]))
    while pi[-1] != 0:
        pi.append(choose(T, p=a[pi[-1], :]))
        x.append(choose(S, p=e[pi[-1]-1, :]))
    pi = [int(p) for p in pi]
    x = [str(q) for q in x]
    return pi, x


In [6]:
# Example 
pi,x = hmm_gen(S, T, a, e)
print(f'length of generated sequence = {len(x)}')
print(f'Hidden state seq: \n {pi}')
print(f'Observable state seq: \n {x}')

length of generated sequence = 117
Hidden state seq: 
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
Observable state seq: 
 ['T', 'H', 'H', 'H', 'H', 'T', 'T', 'H', 'T', 'H', 'T', 'T', 'H', 'T', 'H', 'H', 'H', 'H', 'H', 'T', 'T', 'H', 'T', 'H', 'T', 'H', 'H', 'T', 'H', 'T', 'H', 'T', 'T', 'H', 'H', 'T', 'H', 'H', 'H', 'T', 'T', 'T', 'H', 'H', 'T', 'T', 'T', 'T', 'H', 'T', 'T', 'T', 'H', 'T', 'T', 'T', 'H', 'T', 'H', 'H', 'T', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'T', 'T', 'T', 'H', 'T', 'H', 'T', 'H', 'H', 'T', 'H', 'T', 'T', 'T', 'H', 'T', 'T', 'H', 'H', 'H', 'T', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'T', 'H', 'H', 'T', 'H', 'H', 'H', 'T', 'H', 'T', 'T'

## 2. Viterbi Algorithm

In [7]:
def Viterbi(x, S, T, a, e):#x is obs, h is hid
    obsstates = {S[0]: 0, S[1] : 1}
    v = np.zeros((len(x), T-1))
    v[0, :] = a[0, 1:]
    path = []
    for xi in range(1,len(x)):
        for hi in range(T-1):
            v[xi, hi] = np.max([
                v[xi-1, i] * a[i, hi] * e[hi, obsstates[x[xi]]]
                for i in range(T-1)
            ])
        path.append(np.argmax(v[xi, :]))
    return [int(i) for i in path]

def match(pi1, pi2):
    score = 0
    for i in range(len(pi1)):
        if pi1[i] == pi2[i]:
            score+=1
    return score/len(pi1)

In [8]:
path = Viterbi(x, S, T, a, e)
matchfrac = HMM.match(path, pi)
print(f'{matchfrac*100:.2f}% of the inferred seq matches with the original hidden state sequence.')


83.62% of the inferred seq matches with the original hidden state sequence.


## 3. Forward

In [9]:
def Forward(x, S, T, a, e, matrix=False):
    obs_index = {obs: i for i, obs in enumerate(S)}
    alpha = np.zeros((len(x), T-1))
    for i in range(T-1):
        alpha[0, i] = a[0, i+1] * e[i, obs_index[x[0]]]
    for xi in range(1, len(x)):
        obs_idx = obs_index[x[xi]]
        for hi in range(T-1):
            alpha[xi, hi] = np.sum(
                alpha[xi-1, :] * a[1:, hi+1] * e[hi, obs_idx])
    if matrix == False:
        return np.sum(alpha[-1, :])
    if matrix == True:
        return alpha

In [10]:
f_lk = Forward(x, S, T, a, e)
print(f'The likelihood of input sequence = {f_lk}') 

The likelihood of input sequence = 8.499619488270787e-36


## 4. Backward 

In [11]:
def Backward(x, S, T, a, e, matrix=False):
    obs_index = {s: i for i, s in enumerate(S)}
    N = T - 1
    beta = np.zeros((len(x), T))
    beta[-1, 1:] = 1  
    for xi in reversed(range(len(x)-1)):
        for hi in range(1, T):
            beta[xi, hi] = np.sum(
                beta[xi+1, 1:] * a[hi, 1:] * e[:, obs_index[x[xi+1]]])
    likelihood = np.sum(a[0, 1:] * e[:, obs_index[x[0]]] * beta[0, 1:])
    if matrix == False:
        return likelihood
    if matrix == True:
        return beta

In [12]:
b_lk = Backward(x, S, T, a, e)
print(f'The likelihood of input sequence = {b_lk}') 

The likelihood of input sequence = 8.499619488270795e-36


## 5. Posterior Probability

In [13]:
def ForwardBackward(x, S, T, a, e):
    alpha = Forward(x, S, T, a, e, matrix=True)
    beta = Backward(x, S, T, a, e, matrix=True)

    posterior = np.zeros_like(alpha)
    for xi in range(len(x)):
        ab = alpha[xi] * beta[xi, 1:]
        posterior[xi] = ab / np.sum(ab)
    
    return posterior

In [14]:
post = ForwardBackward(x, S, T, a, e)

#specify the state at which you want to compute the posterior probability
i = 10
print(f'The posterior probabilities of hidden states at {i}:\n p(Fair) = {post[i][0]:.3f} \n p(Biased) = {post[i][1]:.3f}') 

The posterior probabilities of hidden states at 10:
 p(Fair) = 0.892 
 p(Biased) = 0.108
